In [ ]:
!pip install -U scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 87.7 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: Invalid requirement: '==': Expected package name at the start of dependency specifier
    ==
    ^


In [ ]:
!pip install tensorflow==2.15.0

ERROR: Could not find a version that satisfies the requirement tensorflow==2.15.0 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0, 2.19.1, 2.20.0rc0, 2.20.0)
ERROR: No matching distribution found for tensorflow==2.15.0


In [ ]:
!pip install permetrics==1.3.0

In [ ]:
!pip install pandas-ta

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse, r2_score, root_mean_squared_error as rmse
from permetrics.regression import RegressionMetric
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
from pandas import read_csv
from pandas.core.frame import DataFrame
from tensorflow.keras.models import Model, Sequential,load_model
import tensorflow as tf
import os
import numpy as np
from numpy import split, array, dstack
import math
from math import sqrt
import seaborn as sns
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Input, Activation, Flatten, Dropout
from keras.layers import LSTM , GRU
from keras.optimizers import Adam
import csv
from scipy.stats import zscore
from datetime import datetime
import pickle
from keras.models import load_model
from scipy.stats import wilcoxon
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import time
from sklearn.metrics import accuracy_score
from pandas.core.common import flatten
import time
import pandas_ta as ta

In [ ]:
print(tf.__version__)

2.19.0


In [ ]:
models_folder = '/content/drive/MyDrive/KFUPM/Graduation/final_experiments/best_models_us/'
close_index = 3
data_folder = "/content/drive/MyDrive/KFUPM/Graduation/data/Saudi/"

## Functions

### Company data feature engineering functions

In [ ]:
# fill missing values with a value at the same time one day ago
def fill_missing2(values):
 one_day = 1
 for row in range(values.shape[0]):
  for col in range(values.shape[1]):
    if np.isnan(values[row, col]):
     values[row, col] = values[row - one_day, col]

def fill_missing(values):
  return pd.DataFrame(values).fillna(method='ffill').values

def normalized_df(df):
    normalized_df=(df-df.mean())/df.std()
    return normalized_df

#def zlema(src, length):
#    lag = round((length - 1) / 2)
#    ema_data = src + (src - src[lag])
#    zl= ta.ema(ema_data,length = 5)
#    return zl


In [ ]:
def get_company_df(dataset,company_name,start_date):
  company = dataset[(dataset['trading_name '] == company_name)]
  new_date_range = pd.date_range(start=start_date, end="2020-03-05", freq="D")
  company = company.reindex(new_date_range, fill_value=np.nan)
  company = company.drop(["symbol",'name','trading_name ','sectoer'], axis='columns')
  company.replace('?', np.nan, inplace=True)
  company = company.astype('float32')
  company[:] = fill_missing(company.values)
  company['zg'],company['sg'],company['xg'],company['atr'] = ta.aberration(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['lower'],company['mid'],company['upper'] = ta.accbands(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['ad'] = ta.ad(high=company['high'],low= company['low'],close=company['close'],volume=company['volume_traded '],length=5)
  company['adosc'] = ta.adosc(high=company['high'],low= company['low'],volume=company['volume_traded '],close=company['close'],fast=1,slow=5)
  # company['adx'],company['dmp'],company['dmn'] = ta.adx(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['alma'] = ta.alma(company['close'],length=5)
  company['amat1'], company['amat2']= ta.amat(company['close'],length=5)
  # company['ao'] = ta.ao(high=company['high'],low= company['low'],close=company['close'],fast=1,slow=5)
  company['aobv1'],company['aobv2'],company['aobv3'],company['aobv4'],company['aobv5'],company['aobv6'],company['aobv7'] = ta.aobv(company['close'],company['volume_traded '])
  # company['apo'] = ta.apo(company['close'],fast=1,slow=5)
  company['aroon_up'],company['aroon_down'],company['aroon_osc'] = ta.aroon(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['atr']= ta.atr(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['bbands_lower'],company['bbands_mid'],company['bbands_upper'],company['bbands_bandwidth'],company['bbands_percent'] = ta.bbands(company['close'],length=5)
  company['bias'] = ta.bias(company['close'],length=5)
  company['bop']= ta.bop(open_=company['open'],high=company['high'],low= company['low'],close=company['close'],length=5)
  company['ar'],company['br']= ta.brar(open_ = company['open'],high=company['high'],low= company['low'],close=company['close'],length=5)
  company['cci']= ta.cci(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['cfo'] = ta.cfo(company['close'],length=5)
  company['cg'] = ta.cg(company['close'],length=5)
  company['chop']= ta.chop(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['cksp_long'], company['cksp_short']= ta.cksp(high=company['high'],low= company['low'],close=company['close'])
  company['cmf']= ta.cmf(high=company['high'],low= company['low'],close=company['close'],volume = company['volume_traded '],open = company['open'],length=5)
  company['cmo'] = ta.cmo(company['close'],length=5)
  company['coppock'] = ta.coppock(company['close'],length=5,fast=6,slow=11)
  company['cti'] = ta.cti(company['close'],length=5)
  company['decay'] = ta.decay(company['close'],length=5)
  company['decreasing'] = ta.decreasing(company['close'],length=5)
  company['dema'] = ta.dema(company['close'],length=5)
  company['dmp'],company['dmn']= ta.dm(high=company['high'],low= company['low'],length=5)
  company['doncian_lower'],company['doncian_mid'],company['doncian_upper']= ta.donchian(high=company['high'],low= company['low'],lower_length=5,upper_length=5)
  company['dpo'] = ta.dpo(company['close'],length=5)
  company['ebsw'] = ta.ebsw(company['close'],length=5)
  company['efi'] = ta.efi(company['close'],company['volume_traded '],length=5)
  company['ema'] = ta.ema(company['close'],length=5)
  company['entropy'] = ta.entropy(company['close'],length=5)
  company['eom']= ta.eom(high=company['high'],low= company['low'],close=company['close'],volume = company['volume_traded '],length=5)
  company['er'] = ta.er(company['close'],length=5)
  company['eri_bull'],company['eri_bear'] = ta.eri(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['fisher1'],company['fisher2'] = ta.fisher(high=company['high'],low= company['low'],length=5)
  company['fwma'] = ta.fwma(company['close'],length=5)
  company['hilo'],company['hilol'],company['hilos']=ta.hilo(close=company['close'],high=company['high'],low= company['low'],high_length=5,low_length=5)
  company['hma'] = ta.hma(company['close'],length=5)
  company['hl2'] = ta.hl2(company['high'],company['low'])
  company['hlc3'] = ta.hlc3(company['high'],company['low'],company['close'])
  # company['hwm'],company['hwu'],company['hwl'] = ta.hwc(company['close'])
  company['hwma '] = ta.dema(company['close'])
  company['increasing'] = ta.increasing(company['close'],length=5)
  company['inertia'] = ta.inertia(high=company['high'],low= company['low'],close=company['close'],open = company['open'],rvi_length=5,length=5)
  company['jma'] = ta.jma(company['close'],length=5)
  company['kama'] = ta.kama(company['close'],length=5)
  company['kcl'],company['kcb'],company['kcu']= ta.kc(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['kdj1'],company['kdj2'],company['kdj3'] = ta.kdj(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['kurtosis'] = ta.kurtosis(company['close'],length=5)
  company['kvo'],company['kvo_s'] = ta.kvo(high=company['high'],low= company['low'],close=company['close'],volume = company['volume_traded '])
  company['linreg'] = ta.linreg(company['close'],length=5)
  company['log_return'] = ta.log_return(company['close'],length=5)
  company['macd'],company['macd_hist'],company['macd_sig'] = ta.macd(company['close'])
  company['mad'] = ta.mad(company['close'],length=5)
  company['massi'] = ta.massi(high=company['high'],low= company['low'],fast=1,slow=5)
  company['median'] = ta.median(company['close'],length=5)
  company['mfi'] = ta.mfi(high=company['high'],low= company['low'],close=company['close'],volume=company['volume_traded '],length=5)
  company['midpoint'] = ta.midpoint(company['close'],length=5)
  company['midprice'] = ta.midprice(high=company['high'],low= company['low'],length=5)
  company['mom'] = ta.mom(company['close'],length=5)
  company['natr'] = ta.natr(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['nvi'] = ta.nvi(company['close'],company['volume_traded '],length=5)
  company['obv'] = ta.obv(company['close'],company['volume_traded '])
  company['pdist'] = ta.pdist(open_=company['open'],high=company['high'],low= company['low'],close=company['close'])
  company['percent_return'] = ta.percent_return(company['close'],length=5)
  company['pgo'] = ta.pgo(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['psl'] = ta.psl(company['close'],company['open'],length=5)
  # company['pvi'] = ta.pvi(company['close'],company['volume_traded '],length=5)
  company['pvol'] = ta.pvol(company['close'],company['volume_traded '])
  company['pvt'] = ta.pvt(company['close'],company['volume_traded '])
  company['pwma'] = ta.pwma(company['close'],length=5)
  company['qqe'],company['rsi_ma'] ,company['qqel'] ,company['qqes']  = ta.qqe(company['close'],length=5)
  company['qstick'] = ta.qstick(company['open'],company['close'],length=5)
  company['quantile'] = ta.quantile(company['close'],length=5)
  company['rma'] = ta.rma(company['close'],length=5)
  company['roc'] = ta.roc(company['close'],length=5)
  company['rsi'] = ta.rsi(company['close'],length=5)
  company['rsx'] = ta.rsx(company['close'],length=5)
  # company['rvgi1'],company['rvgi2'] = ta.rvgi(open_=company['open'],high=company['high'],low= company['low'],close=company['close'],length=5)
  #company['rvil'],company['rvib'],company['rviu'],company['rvi1'],company['rvi2'],company['rvi3'],company['rvi4'],company['rvi5'],company['rvi6'],company['rvi7'],company['rvi8'],company['rvi9'],company['rvi10'],company['rvi11'] = ta.rvi(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['sinwma'] = ta.sinwma(company['close'],length=5)
  company['skew'] = ta.skew(company['close'],length=5)
  company['slope'] = ta.slope(company['close'],length=5)
  company['sma'] = ta.sma(company['close'],length=5)
  company['smi'],company['smi_s'],company['smi_osc'] = ta.smi(company['close'])
  company['ssf'] = ta.ssf(company['close'],length=5)
  company['stc'],company['stc_macd'],company['stc_stoch'] = ta.stc(company['close'],tclength=5)
  company['stdev'] = ta.stdev(company['close'],length=5)
  # company['stoch_k'],company['stoch_d'] = ta.momentum.stoch(high=company['high'],low= company['low'],close=company['close'])
  company['stochrsi_k'],company['stochrsi_d']= ta.stochrsi(company['close'],length=5)
  company['swma'] = ta.swma(company['close'],length=5)
  company['t3'] = ta.t3(company['close'],length=5)
  company['tema'] = ta.tema(company['close'],length=5)
  company['trima'] = ta.trima(company['close'],length=5)
  company['trix1'],company['trix2']  = ta.trix(company['close'],length=5)
  company['true_range'] = ta.true_range(high=company['high'],low= company['low'],close=company['close'])
  company['tema'] = ta.tema(company['close'],length=5)
  company['ui'] = ta.ui(high=company['high'],close =company['close'],length=5)
  company['uo'] = ta.uo(company['high'],company['low'],company['close'],fast=1,medium=3,slow=5)
  company['variance'] = ta.variance(company['close'],length=5)
  company['vhf'] = ta.vhf(company['close'],length=5)
  company['vidya'] = ta.vidya(company['close'],length=5)
  company['vortex1'],company['vortex2'] = ta.vortex(company['high'],company['low'],company['close'],length=5)
  # company['vp1'],company['vp2'],company['vp3'],company['vp4'],company['vp5'],company['vp6'] = ta.vp(company['close'],company['volume_traded '])
  company['vwap'] = ta.vwap(high=company['high'],low= company['low'],close=company['close'],volume=company['volume_traded '])
  company['vmwa'] = ta.vwma(volume=company['volume_traded '],close=company['close'],length=5)
  company['wcp'] = ta.wcp(high=company['high'],low= company['low'],close=company['close'])
  company['willr'] = ta.willr(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['wma'] = ta.wma(company['close'],length=5)
  company['zlma'] = ta.zlma(company['close'],length=5)
  company['zscore'] = ta.zscore(company['close'],length=5)
  #company['zlema'] = zlema(company['close'],length=5)
  company = company.fillna(0)
  #selected features based on corrlation sheet
  selected_ti = ['open','high','low','close','alma','decay','dema','ema','fwma','hma','hl2','hlc3','hwma ','jma','kama','linreg','median','midpoint','midprice','pwma','quantile','rma','sinwma','sma','ssf','swma','t3','tema','trima','vwap','vmwa','wcp','wma','zlma']
  new_company_df =  company[selected_ti]
  return new_company_df, selected_ti

### Company data prepration functions

In [ ]:
# convert history into inputs and outputs
def to_supervised(X, y, n_input, n_output):
    Xs, ys = [], []
    for i in range(len(X) - n_input - n_output + 1):
        Xs.append(X.iloc[i:i+n_input].values)
        ys.append(y.iloc[i+n_input:i+n_input+n_output].values)
    return np.array(Xs), np.array(ys)

def get_last_val_set_features(data):
  # flatten data
  X = list()
  X.append(data.iloc[len(data)-5:, :])
  return X


In [ ]:
def prepare_company_testing(company,selected_ti, target_col = 'close'):
    # 1. Split data temporally
    # ===============================
    train = company.loc[:'2015-12-31']
    val   = company.loc['2016-01-01':'2016-12-31']
    test  = company.loc['2017-01-01':]

    print(f'Train shape: {train.shape}, Val shape: {val.shape}, Test shape: {test.shape}')

    # 2. Feature scaler (X)
    # ===============================
    feature_scaler = StandardScaler()
    feature_scaler.fit(train[selected_ti])

    test_x_scaled = DataFrame(
        feature_scaler.transform(test[selected_ti]),
        columns=selected_ti,
        index=test.index
    )

    # 3. Target scaler (y)
    # ===============================
    global target_scaler
    target_scaler = StandardScaler()
    target_scaler.fit(train[[target_col]])

    test_y_scaled = DataFrame(
        target_scaler.transform(test[[target_col]]),
        columns=[target_col],
        index=test.index
    )

    print("*******************************************")
    print("Train data after multistep approach")

    print("Test multistep approach")
    test_x, test_y = to_supervised(test_x_scaled, test_y_scaled,
                                   n_input, n_output)
  return test_x,test_y, test


### Company prediction prepration

In [ ]:
def true_y_to_evaluation_form(y_true, n_input, n_out):
  y = list()
  for i in range(len(y_true) - n_input - n_out + 1):
      y.append(y_true.iloc[i+n_input:i+n_input+n_out].values)
  return np.array(y)

# make a forecast
def forecast(model, history):
	yhat = model.predict(history, verbose=0)
	# we only want the vector forecast
	return yhat

# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted,scaler,target_scaler):
  print(f"{len(actual)}, {len(predicted)}")
  predicted_not_normlized = target_scaler.inverse_transform(predicted)
  predicted_not_normlized_rounded = [np.round(num,2) for num in predicted_not_normlized]
  mse_score= mse(actual,predicted_not_normlized_rounded)
  rmse_score = sqrt(mse_score)
  mae_score = mae(actual,predicted_not_normlized_rounded)
  final_r2_score = r2_score(actual,predicted_not_normlized_rounded,multioutput='variance_weighted')
  evaluator = RegressionMetric(actual, predicted_not_normlized_rounded)
  mre_score = np.mean(evaluator.mean_relative_error())
  return mre_score, rmse_score, mae_score, final_r2_score, predicted_not_normlized_rounded

# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts2(actual, predicted):
  mse_score= mse(actual,predicted)
  rmse_score = sqrt(mse_score)
  mae_score = mae(actual,predicted)
  final_r2_score = r2_score(actual,predicted,multioutput='variance_weighted')
  evaluator = RegressionMetric(actual, predicted)
  mre_score = np.mean(evaluator.mean_relative_error())
  return mse_score, rmse_score,mae_score,mre_score,final_r2_score

# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts_mse(actual, predicted):

  mse_list = []
  predicted_not_normlized = target_scaler.inverse_transform(predicted)
  predicted_not_normlized_rounded = [np.round(num,2) for num in predicted_not_normlized]
  for i in range(len(actual)):
    mse_list.append(mse(actual[i],predicted_not_normlized_rounded[i]))
  return mse_list


# summarize scores
def summarize_scores(name, mse_score, rmse_score,mae_score,mre_score,final_r2_score):
  return {  "Model_Name": name,
            "MAE": mae_score,
            "MRE": mre_score,
            "MSE": mse_score,
            "RMSE": rmse_score,
            "R2": final_r2_score}

# evaluate a single model
def evaluate_model(test_x,test_y,model):
  predictions = model.predict(test_x, verbose=0)
  mse_score, rmse_score,mae_score,final_r2_score, y_hat_not_normlized = evaluate_forecasts(test_y, predictions)
  return mse_score, rmse_score,mae_score,final_r2_score,y_hat_not_normlized


def load_models(company_name):
    standalone_models = []
    stacking_models = {}

    # Define the order of the models
    standalone_model_order = ["BRNN", "GRU", "LSTM"]
    standalone_dirname = models_folder + company_name
    stacking_dirname = models_folder + company_name + "/stacking/"

    for filename in os.listdir(standalone_dirname):
        root, ext = os.path.splitext(filename)
        if root.startswith(str(company_name)) and ext == '.pkl':
            with open(os.path.join(standalone_dirname, filename), 'rb') as file:
                print(os.path.join(standalone_dirname, filename))
                model = pickle.load(file)
                standalone_models.append((root, model))  # Store the root and model together
    # Sort the models according to the predefined order
    standalone_models_sorted = sorted(standalone_models, key=lambda x: standalone_model_order.index(x[0].split('_')[-2]))
    # print("**********Models Order***************")
    # [print(x[0]) for x in standalone_models_sorted]
    # print("*************************************")
    # Extract the sorted models

    standalone_models_sorted = [model for _, model in standalone_models_sorted]

    # Load stacking models
    for filename in os.listdir(stacking_dirname):
        root, ext = os.path.splitext(filename)
        if  ext == '.pkl':
            with open(os.path.join(stacking_dirname, filename), 'rb') as file:
                model = pickle.load(file)
                stacking_models[root] = model


    return standalone_models_sorted, stacking_models

# create stacked model input dataset as outputs from the ensemble
def stacked_dataset(members, inputX):
	stackX = None
	for model in members:
		# make prediction
		yhat = model.predict(inputX, verbose=0)
		# stack predictions into [rows, members, probabilities]
		if stackX is None:
			stackX = yhat #
		else:
			stackX = dstack((stackX, yhat))
	# flatten predictions to [rows, members x probabilities]
	#stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
	return stackX

In [ ]:
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

def random_walk_with_drift(input_sequence, forecast_horizon=5):
    """
    input_sequence: list or array of the last w (5) prices
    forecast_horizon: number of steps to forecast (h = 5)
    """
    input_sequence = np.array(input_sequence)
    if len(input_sequence) < 2:
        raise ValueError("Input sequence must have at least two points to compute drift.")

    # Calculate drift as average difference between consecutive values
    drift = np.mean(np.diff(input_sequence))

    # Start from the last value
    last_value = input_sequence[-1]

    # Generate forecast using the drift
    forecast = [last_value + drift * (i + 1) for i in range(forecast_horizon)]

    return forecast

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
def all_models_evaluation_dicts(dataset,selected_companies_dic = {}, company_name = ""):
  company, company_selected_ti = get_company_df(dataset,selected_companies_dic[company_name][0],selected_companies_dic[company_name][1])
  test_x_normlized,test_y_normlized,test = prepare_company_testing(company,company_selected_ti)
  list_res = []
  duration = []
  list_random = []

  test_y_for_evaluation = true_y_to_evaluation_form(test.loc[:,"Close"], 5, 5)


  start_time = time.time()
  # Random Walk
  for window in test_y_for_evaluation:
    list_random.append(random_walk_with_drift(window))
  total_mse_score, rmse_score, mae_score,mre_score ,final_r2_score = evaluate_forecasts2(test_y_for_evaluation, list_random)
  list_res.append(summarize_scores("Random Walk", total_mse_score, rmse_score,mae_score,mre_score,final_r2_score))
  end_time = time.time()
  duration.append(end_time-start_time)
  print(f"Done with Random Walk")

  #Scores of standalone models
  standalone_members,_ = load_models(company_name)
  standalone_model_order = ["BRNN", "GRU", "LSTM"]
  for index_standalone in range(len(standalone_model_order)):
    print(standalone_model_order[index_standalone])
    start_time = time.time()
    predictions = standalone_members[index_standalone].predict(test_x_normlized, verbose=0)
    total_mse_score, rmse_score, mae_score,mre_score ,final_r2_score, forecasts = evaluate_forecasts(test_y_for_evaluation, predictions)
    end_time = time.time()
    duration.append(end_time-start_time)
    list_res.append(summarize_scores(standalone_model_order[index_standalone], total_mse_score, rmse_score,mae_score,mre_score,final_r2_score))
  print(f'Done with Standalone models')

  # Scores of averaging models
  standalone_members_avg,_ = load_models(company_name)
  dict_averaging_models = {
      "averaging_BRNN_GRU_LSTM" : [0,1,2],
      "averaging_BRNN_GRU" : [0,1],
      "averaging_BRNN_LSTM" : [0,2],
      "averaging_GRU_LSTM" : [1,2]
  }
  for key, val in dict_averaging_models.items():
    specified_standalone_members = []
    for model_index in val:
      specified_standalone_members.append(standalone_members_avg[model_index])
    start_time = time.time()
    specified_stacked_test = stacked_dataset(specified_standalone_members, test_x_normlized)
    means_specified_stacked_test = np.mean(specified_stacked_test, axis=2)
    total_mse_score, rmse_score,mae_score,mre_score,final_r2_score,average_forecasts = evaluate_forecasts(test_y_for_evaluation,means_specified_stacked_test)
    end_time = time.time()
    duration.append(end_time-start_time)
    list_res.append(summarize_scores(key, total_mse_score, rmse_score,mae_score,mre_score,final_r2_score))
  print(f'Done with Averaging ensembles')

  # Scores of averaging models
  standalone_members_sta,stacking_members_sta = load_models(company_name)
  dict_stacking_models = {
      "BRNN_GRU_LSTM_meta_learner_ann" : [0,1,2],
      "BRNN_GRU_meta_learner_ann" : [0,1],
      "BRNN_LSTM_meta_learner_ann" : [0,2],
      "GRU_LSTM_meta_learner_ann" : [1,2],
      "BRNN_GRU_LSTM_meta_learner_gru" : [0,1,2],
      "BRNN_GRU_meta_learner_gru" : [0,1],
      "BRNN_LSTM_meta_learner_gru" : [0,2],
      "GRU_LSTM_meta_learner_gru" : [1,2],
      "BRNN_GRU_LSTM_meta_learner_lstm" : [0,1,2],
      "BRNN_GRU_meta_learner_lstm" : [0,1],
      "BRNN_LSTM_meta_learner_lstm" : [0,2],
      "GRU_LSTM_meta_learner_lstm" : [1,2] }
  for key, val in dict_stacking_models.items():
    specified_standalone_members = []
    for model_index in val:
      specified_standalone_members.append(standalone_members_sta[model_index])
    start_time = time.time()
    specified_stacked_test = stacked_dataset(specified_standalone_members, test_x_normlized)
    predictions = stacking_members_sta[key].predict(specified_stacked_test, verbose=0)
    total_mse_score, rmse_score,mae_score,mre_score,final_r2_score,average_forecasts = evaluate_forecasts(test_y_for_evaluation,predictions)
    end_time = time.time()
    duration.append(end_time-start_time)
    list_res.append(summarize_scores(key, total_mse_score, rmse_score,mae_score,mre_score,final_r2_score))
  print(f'Done with Ensemble ensembles')
  return list_res, duration

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%Predictions%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
def all_models_predictions_dicts(dataset,selected_companies_dic,company_name):
  list_pred = []

  company, company_selected_ti = get_company_df(dataset,selected_companies_dic[company_name][0],selected_companies_dic[company_name][1])
  test_x_normlized,test_y_normlized,test = prepare_company_testing(company, company_selected_ti)
  test_y_for_evaluation = true_y_to_evaluation_form(test.iloc[:,close_index], 5, 5)

  list_random = []
  for window in test_y_for_evaluation:
      list_random.append(random_walk_with_drift(window))
  list_pred.append(np.array(list_random))

  standalone_members,stacking_members = load_models(company_name)
  #standalone models
  standalone_model_order = ["BRNN", "GRU", "LSTM"]
  for index_standalone in range(len(standalone_model_order)):
    predictions = standalone_members[index_standalone].predict(test_x_normlized, verbose=0)
    total_mse_score, rmse_score, mae_score,mre_score ,final_r2_score, forecasts = evaluate_forecasts(test_y_for_evaluation, predictions)
    list_pred.append(np.array(forecasts))
  print(f'Done with Standalone models')

  # averaging models
  dict_averaging_models = {
      "averaging_BRNN_GRU_LSTM" : [0,1,2],
      "averaging_BRNN_GRU" : [0,1],
      "averaging_BRNN_LSTM" : [0,2],
      "averaging_GRU_LSTM" : [1,2]
  }
  for key, val in dict_averaging_models.items():
    specified_standalone_members = []
    for model_index in val:
      specified_standalone_members.append(standalone_members[model_index])
    specified_stacked_test = stacked_dataset(specified_standalone_members, test_x_normlized)
    means_specified_stacked_test = np.mean(specified_stacked_test, axis=2)
    total_mse_score, rmse_score, mae_score,mre_score ,final_r2_score, forecasts = evaluate_forecasts(test_y_for_evaluation, means_specified_stacked_test)
    list_pred.append(np.array(forecasts))
  print(f'Done with Averaging ensembles')

  # stacking models
  dict_stacking_models = {
      "BRNN_GRU_LSTM_meta_learner_ann" : [0,1,2],
      "BRNN_GRU_meta_learner_ann" : [0,1],
      "BRNN_LSTM_meta_learner_ann" : [0,2],
      "GRU_LSTM_meta_learner_ann" : [1,2],
      "BRNN_GRU_LSTM_meta_learner_gru" : [0,1,2],
      "BRNN_GRU_meta_learner_gru" : [0,1],
      "BRNN_LSTM_meta_learner_gru" : [0,2],
      "GRU_LSTM_meta_learner_gru" : [1,2],
      "BRNN_GRU_LSTM_meta_learner_lstm" : [0,1,2],
      "BRNN_GRU_meta_learner_lstm" : [0,1],
      "BRNN_LSTM_meta_learner_lstm" : [0,2],
      "GRU_LSTM_meta_learner_lstm" : [1,2] }
  for key, val in dict_stacking_models.items():
    specified_standalone_members = []
    for model_index in val:
      specified_standalone_members.append(standalone_members[model_index])
    specified_stacked_test = stacked_dataset(specified_standalone_members, test_x_normlized)
    predictions = stacking_members[key].predict(specified_stacked_test, verbose=0)
    total_mse_score, rmse_score, mae_score,mre_score ,final_r2_score, forecasts = evaluate_forecasts(test_y_for_evaluation, predictions)
    list_pred.append(np.array(forecasts))
  print(f'Done with Stacking ensembles')

  # Flatten second and third dimensions
  flattened_columns = np.hstack([arr.flatten().reshape(-1, 1) for arr in list_pred])
  return flattened_columns



## Saudi Results

In [ ]:
selected_companies_dic = {"ADC":["ADC","2001-12-31"], "ALRAJHI":["ALRAJHI","2001-12-31"], "GACO":["GACO","2002-01-01"], "EMAAR EC":["EMAAR EC","2006-10-06"], "FITAIHI GROUP":["FITAIHI GROUP","2002-01-26"], "SARCO":["SARCO","2002-01-23"], "SAUDI ELECTRICITY":["SAUDI ELECTRICITY","2002-06-08"], "SPIMACO":["SPIMACO","2001-12-31"], "STC":["STC","2003-01-25"], "TASNEE":["TASNEE","2001-12-31"] }

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/KFUPM/Graduation/data/Saudi/Tadawul_stcks.csv',index_col='date',parse_dates=True)
dataset.sort_values(by='date', inplace = True)

In [ ]:
# Forcasts
for company_key, company_val in selected_companies_dic.items():
  print(f'@@@@@@@@@@@@@@@@@@@@@@@_{company_key}_@@@@@@@@@@@@@@@@@@@@@@@@@@@')
  company, company_selected_ti = get_company_df(dataset,selected_companies_dic[company_key][0],selected_companies_dic[company_key][1])
  company.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/Saudi/{company_key}_data.csv")

@@@@@@@@@@@@@@@@@@@@@@@_ADC_@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@_ALRAJHI_@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@_GACO_@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@_EMAAR EC_@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@_FITAIHI GROUP_@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@_SARCO_@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@_SAUDI ELECTRICITY_@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@_SPIMACO_@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@_STC_@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@_TASNEE_@@@@@@@@@@@@@@@@@@@@@@@@@@@


In [ ]:
company.head()

,open,high,low,close,alma,decay,dema,ema,fwma,hma,...,ssf,swma,t3,tema,trima,vwap,vmwa,wcp,wma,zlma
2001-12-31,2.4,2.4,2.25,2.4,0.0,2.4,0.0,0.0,0.0,0.0,...,2.4,0.0,0.0,0.0,0.0,2.35,0.0,9.450001,0.0,0.0
2002-01-01,2.4,2.4,2.40,2.4,0.0,2.4,0.0,0.0,0.0,0.0,...,2.4,0.0,0.0,0.0,0.0,2.40,0.0,9.600000,0.0,0.0
2002-01-02,2.4,2.4,2.40,2.4,0.0,2.4,0.0,0.0,0.0,0.0,...,2.4,0.0,0.0,0.0,0.0,2.40,0.0,9.600000,0.0,0.0
2002-01-03,2.4,2.4,2.25,2.4,0.0,2.4,0.0,0.0,0.0,0.0,...,2.4,0.0,0.0,0.0,0.0,2.35,0.0,9.450001,0.0,0.0
2002-01-04,2.4,2.4,2.25,2.4,2.4,2.4,2.4,2.4,2.4,0.0,...,2.4,2.4,2.4,2.4,2.4,2.35,2.4,9.450001,2.4,2.4


### Saudi forcasts

In [ ]:
models_names_forcast = models = [
    "Baseline_Random Walk",
    # No Meta Learner (Baseline Models)
    "BRNN", "GRU", "LSTM",

    # Averaging-Based Meta Learners
    "Avg(B,G,L)", "Avg(B,G)", "Avg(B,L)", "Avg(G,L)",

    # Meta Learner: ANN
    "Sta_ANN(B,G,L)", "Sta_ANN(B,G)",
    "Sta_ANN(B,L)", "Sta_ANN(G,L)",

    # Meta Learner: GRU
    "Sta_GRU(B,G,L)", "Sta_GRU(B,G)",
    "Sta_GRU(B,L)", "Sta_GRU(G,L)",

    # Meta Learner: LSTM
    "Sta_LSTM(B,G,L)", "Sta_LSTM(B,G)",
    "Sta_LSTM(B,L)", "Sta_LSTM(G,L)"
]

In [ ]:
# Forcasts
for company_key, company_val in selected_companies_dic.items():
  print(f'@@@@@@@@@@@@@@@@@@@@@@@_{company_key}_@@@@@@@@@@@@@@@@@@@@@@@@@@@')
  forecasts = all_models_predictions_dicts(dataset,selected_companies_dic,company_key)
  df_forecasts = pd.DataFrame(data = forecasts, columns = models_names_forcast )
  df_forecasts.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/Saudi/{company_key}_forcasts_all.csv",index=False)

@@@@@@@@@@@@@@@@@@@@@@@_ADC_@@@@@@@@@@@@@@@@@@@@@@@@@@@
Done with Standalone models
Done with Averaging ensembles
Done with Stacking ensembles
@@@@@@@@@@@@@@@@@@@@@@@_ALRAJHI_@@@@@@@@@@@@@@@@@@@@@@@@@@@
Done with Standalone models
Done with Averaging ensembles
Done with Stacking ensembles
@@@@@@@@@@@@@@@@@@@@@@@_GACO_@@@@@@@@@@@@@@@@@@@@@@@@@@@
Done with Standalone models
Done with Averaging ensembles
Done with Stacking ensembles
@@@@@@@@@@@@@@@@@@@@@@@_EMAAR EC_@@@@@@@@@@@@@@@@@@@@@@@@@@@
Done with Standalone models
Done with Averaging ensembles
Done with Stacking ensembles
@@@@@@@@@@@@@@@@@@@@@@@_FITAIHI GROUP_@@@@@@@@@@@@@@@@@@@@@@@@@@@
Done with Standalone models
Done with Averaging ensembles
Done with Stacking ensembles
@@@@@@@@@@@@@@@@@@@@@@@_SARCO_@@@@@@@@@@@@@@@@@@@@@@@@@@@
Done with Standalone models
Done with Averaging ensembles
Done with Stacking ensembles
@@@@@@@@@@@@@@@@@@@@@@@_SAUDI ELECTRICITY_@@@@@@@@@@@@@@@@@@@@@@@@@@@
Done with Standalone models
Done with Averaging en

In [ ]:
forecasts.shape

(5750, 20)

### Saudi Willxaxon

In [ ]:
models_names_forcast  = [
    "Baseline_Random Walk",
    # No Meta Learner (Baseline Models)
    "BRNN", "GRU", "LSTM",

    # Averaging-Based Meta Learners
    "Avg(B,G,L)", "Avg(B,G)", "Avg(B,L)", "Avg(G,L)",

    # Meta Learner: ANN
    "Sta_ANN(B,G,L)", "Sta_ANN(B,G)",
    "Sta_ANN(B,L)", "Sta_ANN(G,L)",

    # Meta Learner: GRU
    "Sta_GRU(B,G,L)", "Sta_GRU(B,G)",
    "Sta_GRU(B,L)", "Sta_GRU(G,L)",

    # Meta Learner: LSTM
    "Sta_LSTM(B,G,L)", "Sta_LSTM(B,G)",
    "Sta_LSTM(B,L)", "Sta_LSTM(G,L)",

    "Benchmark"
]

In [ ]:
def evaluate_forecasts_mae(actual, predicted):
  mae_list = []
  for i in range(len(actual)):
    mae_list.append(mae(actual[i],predicted[i]))
  return mae_list

def evaluate_forecasts_rmse(actual, predicted):
  rmse_list = []
  for i in range(len(actual)):
    rmse_list.append(rmse(actual[i],predicted[i]))
  return rmse_list

chunk_size = 5
for company_key, company_val in selected_companies_dic.items():
  print(f'@@@@@@@@@@@@@@@@@@@@@@@_{company_key}_@@@@@@@@@@@@@@@@@@@@@@@@@@@')
  df = pd.read_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/Saudi/Forecasts/{company_key}_forcasts_all.csv")
  actual_list = df['Actual'].tolist()
  actual_2d = [actual_list[i:i + chunk_size] for i in range(0, len(actual_list), chunk_size)]
  rmse_df = pd.DataFrame(columns = models_names_forcast)
  for model_name in models_names_forcast:
      forecast_list = df[model_name].tolist()
      forecast_2d = [forecast_list[i:i + chunk_size] for i in range(0, len(forecast_list), chunk_size)]
      rmse_list = evaluate_forecasts_rmse(actual_2d, forecast_2d)
      rmse_df[model_name] = rmse_list
  rmse_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/Saudi/{company_key}_rmse_df.csv",index=False)


@@@@@@@@@@@@@@@@@@@@@@@_ADC_@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@_ALRAJHI_@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@_GACO_@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@_EMAAR EC_@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@_FITAIHI GROUP_@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@_SARCO_@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@_SAUDI ELECTRICITY_@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@_SPIMACO_@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@_STC_@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@_TASNEE_@@@@@@@@@@@@@@@@@@@@@@@@@@@


In [ ]:
def wilcoxonTest(first_model_scores , second_model_scores, first_model_name, second_model_name):
    win , loss = " " , " "
    stat,p= wilcoxon(first_model_scores, second_model_scores)
    alpha = 0.05
    wilcoxonResults = ''
    wilcoxonResNum = 0
    if(p <= alpha):
      mean1 = np.mean(first_model_scores)
      mean2 =  np.mean(second_model_scores)
      if(mean1 < mean2):
        win = first_model_name
        loss = second_model_name
        wilcoxonResNum = 1
      else:
        win = second_model_name
        loss = first_model_name
        wilcoxonResNum = -1
      wilcoxonResults = f"The winner model: {win} , losser model: {loss}. (statics: {stat} , p-Value: {p})"
    else:
      wilcoxonResults = "There is no significance difference"
    return wilcoxonResults, wilcoxonResNum
#%%%%%%%%%%%%%%%%%%%%%%%%%%%WX%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
def construct_wx(models_names):
  wx_list = []
  for company_key in selected_companies_dic.keys():
    print(f'@@@@@@@@@@@@@@@@@@@@@@@_{company_key}_@@@@@@@@@@@@@@@@@@@@@@@@@@@')
    company_results = pd.read_csv(f'/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/Saudi/{company_key}_rmse_df.csv')

    wx = pd.DataFrame(index = models_names, columns = models_names)

    for first_key in models_names:
      for second_key in models_names:
        if(first_key == second_key):
          continue

        first_scores = company_results[first_key].tolist()
        second_scores = company_results[second_key].tolist()

        if(pd.isna(wx.at[first_key, second_key])):
          wx.at[first_key, second_key] = 0

        test_res = wilcoxonTest(first_scores , second_scores,first_key, second_key)[1]
        if(test_res == -1):
          continue
        if(test_res == 1):
          wx.at[first_key, second_key] += 1
    print(wx)

    wx_list.append(wx)
  return wx_list

In [ ]:
#WX
wx_list = construct_wx(models_names_forcast)

@@@@@@@@@@@@@@@@@@@@@@@_ADC_@@@@@@@@@@@@@@@@@@@@@@@@@@@
                     Baseline_Random Walk BRNN  GRU LSTM Avg(B,G,L) Avg(B,G)  \
Baseline_Random Walk                  NaN    0    0    0          0        0   
BRNN                                    1  NaN    0    0          0        0   
GRU                                     1    1  NaN    0          0        0   
LSTM                                    1    1    1  NaN          0        0   
Avg(B,G,L)                              1    1    1    1        NaN        1   
Avg(B,G)                                1    1    1    0          0      NaN   
Avg(B,L)                                1    1    1    0          0        0   
Avg(G,L)                                1    1    1    1          0        0   
Sta_ANN(B,G,L)                          1    0    0    0          0        0   
Sta_ANN(B,G)                            1    1    0    0          0        0   
Sta_ANN(B,L)                            1    1    0    0        

In [ ]:
wx_total = sum(wx_list)
wx_total.head()


,Baseline_Random Walk,BRNN,GRU,LSTM,"Avg(B,G,L)","Avg(B,G)","Avg(B,L)","Avg(G,L)","Sta_ANN(B,G,L)","Sta_ANN(B,G)",...,"Sta_ANN(G,L)","Sta_GRU(B,G,L)","Sta_GRU(B,G)","Sta_GRU(B,L)","Sta_GRU(G,L)","Sta_LSTM(B,G,L)","Sta_LSTM(B,G)","Sta_LSTM(B,L)","Sta_LSTM(G,L)",Benchmark
Baseline_Random Walk,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
BRNN,10,NaN,0,2,0,0,0,0,1,2,...,1,0,1,0,1,1,1,0,1,9
GRU,10,6,NaN,5,0,1,1,2,3,6,...,5,2,1,1,3,0,4,2,3,10
LSTM,10,4,1,NaN,0,1,0,0,2,2,...,3,0,0,0,0,1,2,2,2,9
"Avg(B,G,L)",10,9,6,9,NaN,6,10,6,7,6,...,6,3,2,3,6,4,5,5,4,10


In [ ]:
wx_total.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/Saudi/wx_rmse_Saudi.csv")

### Saudi Time

In [ ]:
models_names_forcast  = [
    "Random Walk",
    # No Meta Learner (Baseline Models)
    "BRNN", "GRU", "LSTM",

    # Averaging-Based Meta Learners
    "Avg(B,G,L)", "Avg(B,G)", "Avg(B,L)", "Avg(G,L)",

    # Meta Learner: ANN
    "Sta_ANN(B,G,L)", "Sta_ANN(B,G)",
    "Sta_ANN(B,L)", "Sta_ANN(G,L)",

    # Meta Learner: GRU
    "Sta_GRU(B,G,L)", "Sta_GRU(B,G)",
    "Sta_GRU(B,L)", "Sta_GRU(G,L)",

    # Meta Learner: LSTM
    "Sta_LSTM(B,G,L)", "Sta_LSTM(B,G)",
    "Sta_LSTM(B,L)", "Sta_LSTM(G,L)"
]
#Time
df_time = pd.DataFrame(index=models_names_forcast)
for company_key, company_val in selected_companies_dic.items():
  print(f'@@@@@@@@@@@@@@@@@@@@@@@_{company_key}_@@@@@@@@@@@@@@@@@@@@@@@@@@@')
  #dataset = pd.read_csv(f'/content/drive/MyDrive/KFUPM/Graduation/data/US/{company_key}_selected_features.csv',index_col='Unnamed: 0',parse_dates=True)
  res,duration = all_models_evaluation_dicts(dataset,selected_companies_dic,company_key)
  #df_res = pd.DataFrame(res)
  df_time[f'{company_key}'] = duration
  #df_res.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/final_results/{company_key}_res_all.csv")
df_time.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/Saudi/time.csv")

@@@@@@@@@@@@@@@@@@@@@@@_ADC_@@@@@@@@@@@@@@@@@@@@@@@@@@@
Done with Random Walk
BRNN
GRU
LSTM
Done with Standalone models
Done with Averaging ensembles
Done with Ensemble ensembles
@@@@@@@@@@@@@@@@@@@@@@@_ALRAJHI_@@@@@@@@@@@@@@@@@@@@@@@@@@@
Done with Random Walk
BRNN
GRU
LSTM
Done with Standalone models
Done with Averaging ensembles
Done with Ensemble ensembles
@@@@@@@@@@@@@@@@@@@@@@@_GACO_@@@@@@@@@@@@@@@@@@@@@@@@@@@
Done with Random Walk
BRNN
GRU
LSTM
Done with Standalone models
Done with Averaging ensembles
Done with Ensemble ensembles
@@@@@@@@@@@@@@@@@@@@@@@_EMAAR EC_@@@@@@@@@@@@@@@@@@@@@@@@@@@
Done with Random Walk
BRNN
GRU
LSTM
Done with Standalone models
Done with Averaging ensembles
Done with Ensemble ensembles
@@@@@@@@@@@@@@@@@@@@@@@_FITAIHI GROUP_@@@@@@@@@@@@@@@@@@@@@@@@@@@
Done with Random Walk
BRNN
GRU
LSTM
Done with Standalone models
Done with Averaging ensembles
Done with Ensemble ensembles
@@@@@@@@@@@@@@@@@@@@@@@_SARCO_@@@@@@@@@@@@@@@@@@@@@@@@@@@
Done with Random Walk
BRNN


## US Results

In [ ]:
us = ["AAPL","BAC","GOOG","JNJ"]

In [ ]:
models_names_forcast  = [
    "Random Walk",
    # No Meta Learner (Baseline Models)
    "BRNN", "GRU", "LSTM",

    # Averaging-Based Meta Learners
    "Avg(B,G,L)", "Avg(B,G)", "Avg(B,L)", "Avg(G,L)",

    # Meta Learner: ANN
    "Sta_ANN(B,G,L)", "Sta_ANN(B,G)",
    "Sta_ANN(B,L)", "Sta_ANN(G,L)",

    # Meta Learner: GRU
    "Sta_GRU(B,G,L)", "Sta_GRU(B,G)",
    "Sta_GRU(B,L)", "Sta_GRU(G,L)",

    # Meta Learner: LSTM
    "Sta_LSTM(B,G,L)", "Sta_LSTM(B,G)",
    "Sta_LSTM(B,L)", "Sta_LSTM(G,L)",
    "Benchmark"
]

In [ ]:
df_time = pd.DataFrame(index=models_names_forcast)
for company_name in us:
  print(f'@@@@@@@@@@@@@@@@@@@@@@@_{company_name}_@@@@@@@@@@@@@@@@@@@@@@@@@@@')
  dataset = pd.read_csv(f'{data_folder}{company_name}_selected_features.csv',index_col='Date',parse_dates=True)
  res,duration = all_models_evaluation_dicts(dataset = dataset,company_name= company_name)
  df_res = pd.DataFrame(res)
  df_res.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/US/final/{company_name}_res_all.csv")
  df_time[f'{company_name}'] = duration
df_time.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/US/final/time.csv")

### Foracsts

In [ ]:
def all_models_predictions_dicts_us(company,company_name):
  list_pred = []
  test_x_normlized,test_y_normlized,test = prepare_company_testing(company,company.columns, 'Close')
  test_y_for_evaluation = true_y_to_evaluation_form(test.iloc[:,close_index], 5, 5)

  list_random = []
  for window in test_y_for_evaluation:
      list_random.append(random_walk_with_drift(window))
  list_pred.append(np.array(list_random))

  standalone_members,stacking_members = load_models(company_name)
  #standalone models
  standalone_model_order = ["BRNN", "GRU", "LSTM"]
  for index_standalone in range(len(standalone_model_order)):
    predictions = standalone_members[index_standalone].predict(test_x_normlized, verbose=0)
    total_mse_score, rmse_score, mae_score,mre_score ,final_r2_score, forecasts = evaluate_forecasts(test_y_for_evaluation, predictions)
    list_pred.append(np.array(forecasts))
  print(f'Done with Standalone models')

  # averaging models
  dict_averaging_models = {
      "averaging_BRNN_GRU_LSTM" : [0,1,2],
      "averaging_BRNN_GRU" : [0,1],
      "averaging_BRNN_LSTM" : [0,2],
      "averaging_GRU_LSTM" : [1,2]
  }
  for key, val in dict_averaging_models.items():
    specified_standalone_members = []
    for model_index in val:
      specified_standalone_members.append(standalone_members[model_index])
    specified_stacked_test = stacked_dataset(specified_standalone_members, test_x_normlized)
    means_specified_stacked_test = np.mean(specified_stacked_test, axis=2)
    total_mse_score, rmse_score, mae_score,mre_score ,final_r2_score, forecasts = evaluate_forecasts(test_y_for_evaluation, means_specified_stacked_test)
    list_pred.append(np.array(forecasts))
  print(f'Done with Averaging ensembles')

  # stacking models
  dict_stacking_models = {
      "BRNN_GRU_LSTM_meta_learner_ann" : [0,1,2],
      "BRNN_GRU_meta_learner_ann" : [0,1],
      "BRNN_LSTM_meta_learner_ann" : [0,2],
      "GRU_LSTM_meta_learner_ann" : [1,2],
      "BRNN_GRU_LSTM_meta_learner_gru" : [0,1,2],
      "BRNN_GRU_meta_learner_gru" : [0,1],
      "BRNN_LSTM_meta_learner_gru" : [0,2],
      "GRU_LSTM_meta_learner_gru" : [1,2],
      "BRNN_GRU_LSTM_meta_learner_lstm" : [0,1,2],
      "BRNN_GRU_meta_learner_lstm" : [0,1],
      "BRNN_LSTM_meta_learner_lstm" : [0,2],
      "GRU_LSTM_meta_learner_lstm" : [1,2] }
  for key, val in dict_stacking_models.items():
    specified_standalone_members = []
    for model_index in val:
      specified_standalone_members.append(standalone_members[model_index])
    specified_stacked_test = stacked_dataset(specified_standalone_members, test_x_normlized)
    predictions = stacking_members[key].predict(specified_stacked_test, verbose=0)
    total_mse_score, rmse_score, mae_score,mre_score ,final_r2_score, forecasts = evaluate_forecasts(test_y_for_evaluation, predictions)
    list_pred.append(np.array(forecasts))
  print(f'Done with Stacking ensembles')

  # Flatten second and third dimensions
  flattened_columns = np.hstack([arr.flatten().reshape(-1, 1) for arr in list_pred])
  return flattened_columns

In [ ]:
# Forcasts
for company_key in us:
  print(f'@@@@@@@@@@@@@@@@@@@@@@@_{company_key}_@@@@@@@@@@@@@@@@@@@@@@@@@@@')
  dataset = pd.read_csv(f'{data_folder}{company_key}_selected_features.csv',index_col='Unnamed: 0',parse_dates=True)
  forecasts = all_models_predictions_dicts_us(dataset,company_key)
  df_forecasts = pd.DataFrame(data = forecasts, columns = models_names_forcast )
  df_forecasts.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/US/{company_key}_forcasts_all.csv",index=False)

@@@@@@@@@@@@@@@@@@@@@@@_AAPL_@@@@@@@@@@@@@@@@@@@@@@@@@@@
/content/drive/MyDrive/KFUPM/Graduation/final_experiments/best_models_us/AAPL/AAPL_BRNN_729.pkl
/content/drive/MyDrive/KFUPM/Graduation/final_experiments/best_models_us/AAPL/AAPL_LSTM_729.pkl
/content/drive/MyDrive/KFUPM/Graduation/final_experiments/best_models_us/AAPL/AAPL_GRU_100.pkl
1150, 1150
1150, 1150
1150, 1150
Done with Standalone models
1150, 1150
1150, 1150
1150, 1150
1150, 1150
Done with Averaging ensembles
1150, 1150
1150, 1150
1150, 1150
1150, 1150
1150, 1150
1150, 1150
1150, 1150
1150, 1150
1150, 1150
1150, 1150
1150, 1150
1150, 1150
Done with Stacking ensembles
@@@@@@@@@@@@@@@@@@@@@@@_BAC_@@@@@@@@@@@@@@@@@@@@@@@@@@@
/content/drive/MyDrive/KFUPM/Graduation/final_experiments/best_models_us/BAC/BAC_BRNN_64.pkl
/content/drive/MyDrive/KFUPM/Graduation/final_experiments/best_models_us/BAC/BAC_LSTM_600.pkl
/content/drive/MyDrive/KFUPM/Graduation/final_experiments/best_models_us/BAC/BAC_GRU_700.pkl
1150, 1150
1150, 1150
11

### Wilxaon

In [ ]:
models_names_forcast  = [
    "Baseline_Random Walk",
    # No Meta Learner (Baseline Models)
    "BRNN", "GRU", "LSTM",

    # Averaging-Based Meta Learners
    "Avg(B,G,L)", "Avg(B,G)", "Avg(B,L)", "Avg(G,L)",

    # Meta Learner: ANN
    "Sta_ANN(B,G,L)", "Sta_ANN(B,G)",
    "Sta_ANN(B,L)", "Sta_ANN(G,L)",

    # Meta Learner: GRU
    "Sta_GRU(B,G,L)", "Sta_GRU(B,G)",
    "Sta_GRU(B,L)", "Sta_GRU(G,L)",

    # Meta Learner: LSTM
    "Sta_LSTM(B,G,L)", "Sta_LSTM(B,G)",
    "Sta_LSTM(B,L)", "Sta_LSTM(G,L)",
    "Benchmark"
]

def wilcoxonTest(first_model_scores , second_model_scores, first_model_name, second_model_name):
    win , loss = " " , " "
    stat,p= wilcoxon(first_model_scores, second_model_scores)
    alpha = 0.05
    wilcoxonResults = ''
    wilcoxonResNum = 0
    if(p <= alpha):
      mean1 = np.mean(first_model_scores)
      mean2 =  np.mean(second_model_scores)
      if(mean1 < mean2):
        win = first_model_name
        loss = second_model_name
        wilcoxonResNum = 1
      else:
        win = second_model_name
        loss = first_model_name
        wilcoxonResNum = -1
      wilcoxonResults = f"The winner model: {win} , losser model: {loss}. (statics: {stat} , p-Value: {p})"
    else:
      wilcoxonResults = "There is no significance difference"
    return wilcoxonResults, wilcoxonResNum
#%%%%%%%%%%%%%%%%%%%%%%%%%%%WX%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
def construct_wx(models_names, companies):
  wx_list = []
  for company_key in companies:
    print(f'@@@@@@@@@@@@@@@@@@@@@@@_{company_key}_@@@@@@@@@@@@@@@@@@@@@@@@@@@')
    company_results = pd.read_csv(f'/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/US/{company_key}_rmse_df.csv')

    wx = pd.DataFrame(index = models_names, columns = models_names)

    for first_key in models_names:
      for second_key in models_names:
        if(first_key == second_key):
          continue

        first_scores = company_results[first_key].tolist()
        second_scores = company_results[second_key].tolist()

        if(pd.isna(wx.at[first_key, second_key])):
          wx.at[first_key, second_key] = 0

        test_res = wilcoxonTest(first_scores , second_scores,first_key, second_key)[1]
        if(test_res == -1):
          continue
        if(test_res == 1):
          wx.at[first_key, second_key] += 1
    print(wx)

    wx_list.append(wx)
  return wx_list

In [ ]:
def evaluate_forecasts_rmse_us(actual, predicted):
  rmse_list = []
  for i in range(len(actual)):
    rmse_list.append(rmse(actual[i],predicted[i]))
  return rmse_list

chunk_size = 5
for company_key in us:
  print(f'@@@@@@@@@@@@@@@@@@@@@@@_{company_key}_@@@@@@@@@@@@@@@@@@@@@@@@@@@')
  df = pd.read_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/US/{company_key}_forcasts_all.csv")
  actual_list = df['Actual'].tolist()
  actual_2d = [actual_list[i:i + chunk_size] for i in range(0, len(actual_list), chunk_size)]
  rmse_df = pd.DataFrame(columns = models_names_forcast)
  for model_name in models_names_forcast:
      forecast_list = df[model_name].tolist()
      forecast_2d = [forecast_list[i:i + chunk_size] for i in range(0, len(forecast_list), chunk_size)]
      rmse_list = evaluate_forecasts_rmse_us(actual_2d, forecast_2d)
      rmse_df[model_name] = rmse_list
  rmse_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/US/{company_key}_rmse_df.csv",index=False)


@@@@@@@@@@@@@@@@@@@@@@@_AAPL_@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@_BAC_@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@_GOOG_@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@_JNJ_@@@@@@@@@@@@@@@@@@@@@@@@@@@


In [ ]:
#WX
wx_list = construct_wx(models_names_forcast,us)

@@@@@@@@@@@@@@@@@@@@@@@_AAPL_@@@@@@@@@@@@@@@@@@@@@@@@@@@
                     Baseline_Random Walk BRNN  GRU LSTM Avg(B,G,L) Avg(B,G)  \
Baseline_Random Walk                  NaN    0    0    0          0        0   
BRNN                                    1  NaN    0    1          0        0   
GRU                                     1    0  NaN    1          0        0   
LSTM                                    1    0    0  NaN          0        0   
Avg(B,G,L)                              1    0    0    1        NaN        0   
Avg(B,G)                                1    1    1    1          1      NaN   
Avg(B,L)                                1    0    0    1          0        0   
Avg(G,L)                                1    0    0    1          0        0   
Sta_ANN(B,G,L)                          1    1    1    1          1        0   
Sta_ANN(B,G)                            1    0    0    1          0        0   
Sta_ANN(B,L)                            1    1    0    1       

In [ ]:
wx_total = sum(wx_list)
wx_total.head()


,Baseline_Random Walk,BRNN,GRU,LSTM,"Avg(B,G,L)","Avg(B,G)","Avg(B,L)","Avg(G,L)","Sta_ANN(B,G,L)","Sta_ANN(B,G)",...,"Sta_ANN(G,L)","Sta_GRU(B,G,L)","Sta_GRU(B,G)","Sta_GRU(B,L)","Sta_GRU(G,L)","Sta_LSTM(B,G,L)","Sta_LSTM(B,G)","Sta_LSTM(B,L)","Sta_LSTM(G,L)",Benchmark
Baseline_Random Walk,NaN,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BRNN,3,NaN,1,3,1,1,2,2,1,1,...,1,1,1,1,1,1,1,2,1,2
GRU,4,1,NaN,3,1,1,1,2,2,1,...,1,1,2,2,3,2,2,3,2,3
LSTM,4,1,0,NaN,0,1,1,0,2,1,...,1,2,1,2,1,2,2,2,1,3
"Avg(B,G,L)",4,1,1,3,NaN,1,3,3,2,1,...,1,0,1,2,1,2,2,3,2,3


In [ ]:
wx_total.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/US/wx_US.csv")

# Benchmark Saudi


##Benchmark model replication

### Company data feature engineering functions

In [ ]:
# fill missing values with a value at the same time one day ago
def fill_missing(values):
 one_day = 1
 for row in range(values.shape[0]):
  for col in range(values.shape[1]):
    if np.isnan(values[row, col]):
     values[row, col] = values[row - one_day, col]

def normalized_df(df):
    normalized_df=(df-df.mean())/df.std()
    return normalized_df

def zlema(src, length):
   lag = round((length - 1) / 2)
   ema_data = src + (src - src[lag])
   zl= ta.ema(ema_data,length = 5)
   return zl



In [ ]:
def get_company_df_bench(dataset,company_name,start_date):
  company = dataset[(dataset['trading_name '] == company_name)]
  new_date_range = pd.date_range(start=start_date, end="2020-03-05", freq="D")
  company = company.reindex(new_date_range, fill_value=np.NaN)
  company = company.drop(["symbol",'name','trading_name ','sectoer'], axis='columns')
  company.replace('?', np.NaN, inplace=True)
  company = company.astype('float32')
  fill_missing(company.values)
  values = company.values
  company['dema'] = ta.dema(company['close'],length=5)
  company['hma'] = ta.hma(company['close'],length=5)
  company['tema'] = ta.tema(company['close'],length=5)
  company['wma'] = ta.wma(company['close'],length=5)
  #company['zlma'] = zlema(company['close'],length=5)
  company['zlema'] = zlema(company['close'],length=5)
  company = company.fillna(0)
  #selected features based on corrlation sheet
  selected_ti = ['open','high','low','close','change','perc_Change','volume_traded ',"value_traded",'no_trades ','dema','hma','tema','wma','zlema']
  new_company_df =  company[selected_ti]
  return new_company_df, selected_ti

### Company data prepration functions

In [ ]:
# convert history into inputs and outputs
def to_supervised(data, n_input, n_out):
  # flatten data
  X, y = list(), list()
  in_start = 0
	# step over the entire history one time step at a time
  for _ in range(len(data)):
    # define the end of the input sequence
    in_end = in_start + n_input
    out_end = in_end + n_out
    # ensure we have enough data for this instance
    if out_end < len(data):
      X.append(data.iloc[in_start:in_end, :])
      y.append(data.iloc[in_end:out_end, 3])
    # move along one time step
    in_start += 1
  print(f'X shape: {array(X).shape}')
  print(f'y shape: {array(y).shape}')
  return array(X), array(y)

def get_last_val_set_features(data):
  # flatten data
  X = list()
  X.append(data.iloc[len(data)-5:, :])
  return X


In [ ]:
#Test starting from 1-1-2017 based on 'Present Effictive short-term forecasts of Saudi stock price trends using technical indicators and large-scale multivariate time series'
# split a multivariate dataset into train/test sets
def prepare_company_for_training_and_testing(company,selected_ti):
  train,val, test =  company.loc[:'2015-12-31'],company.loc['1-1-2016':'31-12-2016'],company.loc['1-1-2017':]
  print(f'Train shape: {train.shape}, Val shape: {val.shape}, Test shape: {test.shape}')
  scaler = MinMaxScaler()
  scaler.fit(train)
  data = scaler.transform(train)
  train_normlized = DataFrame(data = data , columns = selected_ti ,index=train.index)
  scaler = MinMaxScaler()
  scaler.fit(val)
  data = scaler.transform(val)
  val_normlized = DataFrame(data = data , columns = selected_ti ,index=val.index)
  scaler = MinMaxScaler()
  scaler.fit(test)
  data = scaler.transform(test)
  test_normlized = DataFrame(data = data , columns =selected_ti,index=test.index)
  print("*******************************************")
  print("Train data after multistep approach")
  train_x_normlized, train_y_normlized = to_supervised(train_normlized, 5,5)
  print("Validation multistep approach")
  val_x_normlized,val_y_normlized = to_supervised(val_normlized, 5,5)
  print("Test multistep approach")
  last_val = get_last_val_set_features(val_normlized)
  test_x_normlized,test_y_normlized = to_supervised(test_normlized, 5,5)
  return train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized, test

In [ ]:
def true_y_to_evaluation_form(y_true, n_input, n_out):
  y = list()
  in_start = 0
	# step over the entire history one time step at a time
  for _ in range(len(y_true)):
    # define the end of the input sequence
    in_end = in_start + n_input
    out_end = in_end + n_out
    # ensure we have enough data for this instance
    if out_end < len(y_true):
      y.append(y_true[in_end:out_end])
    # move along one time step
    in_start += 1
  return array(y)


# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted):
  scaler = MinMaxScaler()
  scaler.fit(actual)
  predicted_not_normlized = scaler.inverse_transform(predicted)
  predicted_not_normlized_rounded = [np.round(num,2) for num in predicted_not_normlized]
  mse_score= mse(actual,predicted_not_normlized_rounded)
  rmse_score = sqrt(mse_score)
  mae_score = mae(actual,predicted_not_normlized_rounded)
  final_r2_score = r2_score(actual,predicted_not_normlized_rounded,multioutput='variance_weighted')
  evaluator = RegressionMetric(actual, predicted_not_normlized_rounded)
  return mse_score, rmse_score,mae_score,final_r2_score,predicted_not_normlized_rounded

# evaluate a single model
def evaluate_model(test_x,test_y,model):
  predictions = model.predict(test_x, verbose=0)
  mse_score, rmse_score,mae_score,final_r2_score, y_hat_not_normlized = evaluate_forecasts(test_y, predictions)
  return mse_score, rmse_score,mae_score,final_r2_score,y_hat_not_normlized

def summarize_scores2(name, mse_score, rmse_score,mae_score,final_r2_score):
  return {  "Model_Name": name,
            "MAE": mae_score,
            "MSE": mse_score,
            "RMSE": rmse_score,
            "R2": final_r2_score}

### model

In [ ]:
from tensorflow.keras.optimizers import Nadam

In [ ]:
def get_gru_models_info(train_x,train_y,val_x,val_y,test_x,y_test,test_y_for_evaluation,company_name):
  # define parameters
  verbose, epochs, batch_size = 0, 20, math.floor(len(train_x)*0.05)
  n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
  # reshape output into [samples, timesteps, features]
  train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))
  # define model
  bench_gru = Sequential()
  bench_gru.add(GRU(256, activation='tanh', input_shape=(n_timesteps, n_features)))
  bench_gru.add(Dense(5))
  bench_gru.compile(loss='mse', optimizer=Nadam())
  tf.keras.utils.plot_model(bench_gru)
  # fit network
  bench_gru.fit(train_x,
          train_y,
          epochs=epochs,
          batch_size=batch_size,
          verbose=verbose,
          validation_data=(val_x,val_y))
  start_time = time.time()
  predictions = bench_gru.predict(test_x, verbose=0)
  mse_score_G_p, rmse_score_G_p,mae_score_G_p,final_r2_score_G_p,predictions_G_p = evaluate_model(test_x,test_y_for_evaluation,bench_gru)
  end_time = time.time()
  df_time.at["Bench",company_name] = end_time - start_time
  print(f"{company_name} GRU(256)")
  #pickle.dump(bench_gru , open(f'/content/drive/MyDrive/KFUPM/Graduation/final_experiments/models/Bench/{company_name}_Benc_GRU_{256}.pkl' , 'wb'))
  #df_forecasts = pd.DataFrame(data = np.array(predictions_G_p).flatten() )
  #df_forecasts.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/models/Bench/{company_name}_Benc_forcasts_all.csv")
  summarize_scores2('GRU', mse_score_G_p, rmse_score_G_p,mae_score_G_p,final_r2_score_G_p)




In [ ]:
df_time = pd.read_csv("/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/Saudi/time.csv")
def train_evalute_models_for_company(dataset,selected_companies_dic):
  for company_key, company_val in selected_companies_dic.items():
    print(f'@@@@@@@@@@@@@@@@@@@@@@@_{company_key}_@@@@@@@@@@@@@@@@@@@@@@@@@@@')
    company, company_selected_ti = get_company_df_bench(dataset,selected_companies_dic[company_key][0],selected_companies_dic[company_key][1])
    train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test = prepare_company_for_training_and_testing(company,company_selected_ti)
    test_y_for_evaluation = true_y_to_evaluation_form(test['close'].values, 5, 5)
    get_gru_models_info(train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test_y_for_evaluation,company_key)


In [ ]:
df_time.tail()

In [ ]:
df_time.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/Saudi/time_bench.csv")

### Excution

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/KFUPM/Graduation/data/Saudi/Tadawul_stcks.csv',index_col='date',parse_dates=True)
dataset.sort_values(by='date', inplace = True)

In [ ]:
dataset.head()

In [ ]:
selected_companies_dic = {"ADC":["ADC","2001-12-31"], "ALRAJHI":["ALRAJHI","2001-12-31"], "GACO":["GACO","2002-01-01"], "EMAAR EC":["EMAAR EC","2006-10-06"], "FITAIHI GROUP":["FITAIHI GROUP","2002-01-26"], "SARCO":["SARCO","2002-01-23"], "SAUDI ELECTRICITY":["SAUDI ELECTRICITY","2002-06-08"], "SPIMACO":["SPIMACO","2001-12-31"], "STC":["STC","2003-01-25"], "TASNEE":["TASNEE","2001-12-31"] }

In [ ]:
train_evalute_models_for_company(dataset,selected_companies_dic)

@@@@@@@@@@@@@@@@@@@@@@@_ADC_@@@@@@@@@@@@@@@@@@@@@@@@@@@
Train shape: (5114, 14), Val shape: (366, 14), Test shape: (1160, 14)
*******************************************
Train data after multistep approach
X shape: (5104, 5, 14)
y shape: (5104, 5)
Validation multistep approach
X shape: (356, 5, 14)
y shape: (356, 5)
Test multistep approach
X shape: (1150, 5, 14)
y shape: (1150, 5)
ADC GRU(256)
@@@@@@@@@@@@@@@@@@@@@@@_ALRAJHI_@@@@@@@@@@@@@@@@@@@@@@@@@@@
Train shape: (5114, 14), Val shape: (366, 14), Test shape: (1160, 14)
*******************************************
Train data after multistep approach
X shape: (5104, 5, 14)
y shape: (5104, 5)
Validation multistep approach
X shape: (356, 5, 14)
y shape: (356, 5)
Test multistep approach
X shape: (1150, 5, 14)
y shape: (1150, 5)
ALRAJHI GRU(256)
@@@@@@@@@@@@@@@@@@@@@@@_GACO_@@@@@@@@@@@@@@@@@@@@@@@@@@@
Train shape: (5113, 14), Val shape: (366, 14), Test shape: (1160, 14)
*******************************************
Train data after multistep 

## Combine the benchmark with our models' forcasts

In [ ]:
def prepare_testing_values(company_name,start_date):
  company = dataset[(dataset['trading_name '] == company_name)]
  new_date_range = pd.date_range(start=start_date, end="2020-03-05", freq="D")
  company = company.reindex(new_date_range, fill_value=np.NaN)
  company = company.drop(["symbol",'name','trading_name ','sectoer'], axis='columns')
  company.replace('?', np.NaN, inplace=True)
  company = company.astype('float32')
  fill_missing(company.values)
  values = company.values
  train,val, test =  company.loc[:'2015-12-31'],company.loc['1-1-2016':'31-12-2016'],company.loc['1-1-2017':]
  print(f'Train shape: {train.shape}, Val shape: {val.shape}, Test shape: {test.shape}')
  test_y_for_evaluation = np.array(true_y_to_evaluation_form(test['close'].values, 5, 5)).flatten()
  return test_y_for_evaluation

In [ ]:
selected_companies_dic = {"ADC":["ADC","2001-12-31"], "ALRAJHI":["ALRAJHI","2001-12-31"], "GACO":["GACO","2002-01-01"], "EMAAR EC":["EMAAR EC","2006-10-06"], "FITAIHI GROUP":["FITAIHI GROUP","2002-01-26"], "SARCO":["SARCO","2002-01-23"], "SAUDI ELECTRICITY":["SAUDI ELECTRICITY","2002-06-08"], "SPIMACO":["SPIMACO","2001-12-31"], "STC":["STC","2003-01-25"], "TASNEE":["TASNEE","2001-12-31"] }

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/KFUPM/Graduation/data/Saudi/Tadawul_stcks.csv',index_col='date',parse_dates=True)
dataset.sort_values(by='date', inplace = True)

In [ ]:
for company_key, company_val in selected_companies_dic.items():
  print(f'@@@@@@@@@@@@@@@@@@@@@@@_{company_key}_@@@@@@@@@@@@@@@@@@@@@@@@@@@')
  models = pd.read_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/Saudi/{company_key}_forcasts_all.csv")
  bench = pd.read_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/models/Bench/{company_key}_Benc_forcasts_all.csv")
  #models["Benchmark"] = bench["0"]
  #models["Actual"] = prepare_testing_values(company_key,company_val[1])
  models.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/Saudi/Forecasts/{company_key}_forcasts_all.csv",index=False)



@@@@@@@@@@@@@@@@@@@@@@@_ADC_@@@@@@@@@@@@@@@@@@@@@@@@@@@
Train shape: (5114, 9), Val shape: (366, 9), Test shape: (1160, 9)
@@@@@@@@@@@@@@@@@@@@@@@_ALRAJHI_@@@@@@@@@@@@@@@@@@@@@@@@@@@
Train shape: (5114, 9), Val shape: (366, 9), Test shape: (1160, 9)
@@@@@@@@@@@@@@@@@@@@@@@_GACO_@@@@@@@@@@@@@@@@@@@@@@@@@@@
Train shape: (5113, 9), Val shape: (366, 9), Test shape: (1160, 9)
@@@@@@@@@@@@@@@@@@@@@@@_EMAAR EC_@@@@@@@@@@@@@@@@@@@@@@@@@@@
Train shape: (3374, 9), Val shape: (366, 9), Test shape: (1160, 9)
@@@@@@@@@@@@@@@@@@@@@@@_FITAIHI GROUP_@@@@@@@@@@@@@@@@@@@@@@@@@@@
Train shape: (5088, 9), Val shape: (366, 9), Test shape: (1160, 9)
@@@@@@@@@@@@@@@@@@@@@@@_SARCO_@@@@@@@@@@@@@@@@@@@@@@@@@@@
Train shape: (5091, 9), Val shape: (366, 9), Test shape: (1160, 9)
@@@@@@@@@@@@@@@@@@@@@@@_SAUDI ELECTRICITY_@@@@@@@@@@@@@@@@@@@@@@@@@@@
Train shape: (4955, 9), Val shape: (366, 9), Test shape: (1160, 9)
@@@@@@@@@@@@@@@@@@@@@@@_SPIMACO_@@@@@@@@@@@@@@@@@@@@@@@@@@@
Train shape: (5114, 9), Val shape: (366, 9)

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/Saudi/Forecasts/ADC_forcasts_all.csv")

In [ ]:
df.shape

(5750, 22)

In [ ]:
df.head()

,Baseline_Random Walk,BRNN,GRU,LSTM,"Avg(B,G,L)","Avg(B,G)","Avg(B,L)","Avg(G,L)","Sta_ANN(B,G,L)","Sta_ANN(B,G)",...,"Sta_GRU(B,G,L)","Sta_GRU(B,G)","Sta_GRU(B,L)","Sta_GRU(G,L)","Sta_LSTM(B,G,L)","Sta_LSTM(B,G)","Sta_LSTM(B,L)","Sta_LSTM(G,L)",Benchmark,Actual
0,13.389999,13.71,13.81,13.63,13.72,13.76,13.67,13.72,13.96,14.03,...,13.94,13.93,13.88,13.81,13.94,13.97,13.91,13.98,13.68,13.84
1,13.299999,13.77,13.96,13.68,13.81,13.87,13.73,13.82,13.99,14.02,...,13.94,13.95,13.87,13.91,13.94,13.96,13.95,13.98,14.41,13.84
2,13.209999,13.69,13.97,13.68,13.78,13.83,13.68,13.82,14.00,14.00,...,13.91,13.93,13.87,13.97,13.97,14.01,13.94,14.01,14.58,14.16
3,13.119999,13.76,14.00,13.76,13.84,13.88,13.76,13.88,14.00,14.03,...,13.95,14.00,13.91,13.93,13.94,14.04,13.95,13.97,14.65,12.75
4,13.029999,13.76,14.00,13.69,13.81,13.88,13.72,13.84,13.98,14.05,...,14.04,13.98,13.93,13.91,13.95,14.04,14.01,13.99,14.98,13.48


# Benchmark US


##Benchmark model replication

### Company data feature engineering functions

In [ ]:
# fill missing values with a value at the same time one day ago
def fill_missing(values):
 one_day = 1
 for row in range(values.shape[0]):
  for col in range(values.shape[1]):
    if np.isnan(values[row, col]):
     values[row, col] = values[row - one_day, col]

def normalized_df(df):
    normalized_df=(df-df.mean())/df.std()
    return normalized_df

def ema(prices, period):
    ema = []
    k = 2 / (period + 1)
    ema_val = sum(prices[:period]) / period  # أول قيمة = SMA
    for i in range(len(prices)):
        if i < period:
            ema.append(None)
        elif i == period:
            ema.append(ema_val)
        else:
            ema_val = prices[i] * k + ema_val * (1 - k)
            ema.append(ema_val)
    return ema

def zlema(src, length):
   lag = round((length - 1) / 2)
   ema_data = src + (src - src[lag])
   zl= ta.ema(ema_data,  5)
   return zl



In [ ]:
def get_company_df_bench_us(company,company_name):
  company = company.fillna(0)
  #company = fill_missing(dataset.values)
  #company['dema'] = ta.dema(company['Close'],length=5)
  #company['hma'] = ta.hma(company['Close'],length=5)
  #company['tema'] = ta.tema(company['Close'],length=5)
  #company['wma'] = ta.wma(company['Close'],length=5)
  #company['zlma'] = zlema(company['Close'],length=5)
  #company['zlema'] = zlema(company['Close'],length=5)
  #selected features based on corrlation sheet
  selected_ti = ['Open','High','Low','Close','dema','hma','tema','wma','zelma']
  new_company_df =  company[selected_ti]
  return new_company_df, selected_ti

### Company data prepration functions

In [ ]:
# convert history into inputs and outputs
def to_supervised(data, n_input, n_out):
  # flatten data
  X, y = list(), list()
  in_start = 0
	# step over the entire history one time step at a time
  for _ in range(len(data)):
    # define the end of the input sequence
    in_end = in_start + n_input
    out_end = in_end + n_out
    # ensure we have enough data for this instance
    if out_end < len(data):
      X.append(data.iloc[in_start:in_end, :])
      y.append(data.iloc[in_end:out_end, close_index])
    # move along one time step
    in_start += 1
  print(f'X shape: {array(X).shape}')
  print(f'y shape: {array(y).shape}')
  return array(X), array(y)

def get_last_val_set_features(data):
  # flatten data
  X = list()
  X.append(data.iloc[len(data)-5:, :])
  return X


In [ ]:
#Test starting from 1-1-2017 based on 'Present Effictive short-term forecasts of Saudi stock price trends using technical indicators and large-scale multivariate time series'
# split a multivariate dataset into train/test sets
def prepare_company_for_training_and_testing(company,selected_ti):
  company = company.sort_index()
  train,val, test =  company.loc[:'2015-12-31'],company.loc['1-1-2016':'31-12-2016'],company.loc['1-1-2017':]
  print(f'Train shape: {train.shape}, Val shape: {val.shape}, Test shape: {test.shape}')
  scaler = MinMaxScaler()
  scaler.fit(train)
  data = scaler.transform(train)
  train_normlized = DataFrame(data = data , columns = selected_ti ,index=train.index)
  scaler = MinMaxScaler()
  scaler.fit(val)
  data = scaler.transform(val)
  val_normlized = DataFrame(data = data , columns = selected_ti ,index=val.index)
  scaler = MinMaxScaler()
  scaler.fit(test)
  data = scaler.transform(test)
  test_normlized = DataFrame(data = data , columns =selected_ti,index=test.index)
  print("*******************************************")
  print("Train data after multistep approach")
  train_x_normlized, train_y_normlized = to_supervised(train_normlized, 5,5)
  print("Validation multistep approach")
  val_x_normlized,val_y_normlized = to_supervised(val_normlized, 5,5)
  print("Test multistep approach")
  last_val = get_last_val_set_features(val_normlized)
  test_x_normlized,test_y_normlized = to_supervised(test_normlized, 5,5)
  return train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized, test

def all_models_predictions_dicts_us(company,company_name):
  list_pred = []
  test_x_normlized,test_y_normlized,test = prepare_company_testing(company,company.columns, 'Close')
  test_y_for_evaluation = true_y_to_evaluation_form(test.iloc[:,close_index], 5, 5)

  list_random = []
  for window in test_y_for_evaluation:
      list_random.append(random_walk_with_drift(window))
  list_pred.append(np.array(list_random))

  standalone_members,stacking_members = load_models(company_name)
  #standalone models
  standalone_model_order = ["BRNN", "GRU", "LSTM"]
  for index_standalone in range(len(standalone_model_order)):
    predictions = standalone_members[index_standalone].predict(test_x_normlized, verbose=0)
    total_mse_score, rmse_score, mae_score,mre_score ,final_r2_score, forecasts = evaluate_forecasts(test_y_for_evaluation, predictions)
    list_pred.append(np.array(forecasts))
  print(f'Done with Standalone models')

  # averaging models
  dict_averaging_models = {
      "averaging_BRNN_GRU_LSTM" : [0,1,2],
      "averaging_BRNN_GRU" : [0,1],
      "averaging_BRNN_LSTM" : [0,2],
      "averaging_GRU_LSTM" : [1,2]
  }
  for key, val in dict_averaging_models.items():
    specified_standalone_members = []
    for model_index in val:
      specified_standalone_members.append(standalone_members[model_index])
    specified_stacked_test = stacked_dataset(specified_standalone_members, test_x_normlized)
    means_specified_stacked_test = np.mean(specified_stacked_test, axis=2)
    total_mse_score, rmse_score, mae_score,mre_score ,final_r2_score, forecasts = evaluate_forecasts(test_y_for_evaluation, means_specified_stacked_test)
    list_pred.append(np.array(forecasts))
  print(f'Done with Averaging ensembles')

  # stacking models
  dict_stacking_models = {
      "BRNN_GRU_LSTM_meta_learner_ann" : [0,1,2],
      "BRNN_GRU_meta_learner_ann" : [0,1],
      "BRNN_LSTM_meta_learner_ann" : [0,2],
      "GRU_LSTM_meta_learner_ann" : [1,2],
      "BRNN_GRU_LSTM_meta_learner_gru" : [0,1,2],
      "BRNN_GRU_meta_learner_gru" : [0,1],
      "BRNN_LSTM_meta_learner_gru" : [0,2],
      "GRU_LSTM_meta_learner_gru" : [1,2],
      "BRNN_GRU_LSTM_meta_learner_lstm" : [0,1,2],
      "BRNN_GRU_meta_learner_lstm" : [0,1],
      "BRNN_LSTM_meta_learner_lstm" : [0,2],
      "GRU_LSTM_meta_learner_lstm" : [1,2] }
  for key, val in dict_stacking_models.items():
    specified_standalone_members = []
    for model_index in val:
      specified_standalone_members.append(standalone_members[model_index])
    specified_stacked_test = stacked_dataset(specified_standalone_members, test_x_normlized)
    predictions = stacking_members[key].predict(specified_stacked_test, verbose=0)
    total_mse_score, rmse_score, mae_score,mre_score ,final_r2_score, forecasts = evaluate_forecasts(test_y_for_evaluation, predictions)
    list_pred.append(np.array(forecasts))
  print(f'Done with Stacking ensembles')

  # Flatten second and third dimensions
  flattened_columns = np.hstack([arr.flatten().reshape(-1, 1) for arr in list_pred])
  return flattened_columns

In [ ]:
def true_y_to_evaluation_form(y_true, n_input, n_out):
  y = list()
  in_start = 0
	# step over the entire history one time step at a time
  for _ in range(len(y_true)):
    # define the end of the input sequence
    in_end = in_start + n_input
    out_end = in_end + n_out
    # ensure we have enough data for this instance
    if out_end < len(y_true):
      y.append(y_true[in_end:out_end])
    # move along one time step
    in_start += 1
  return array(y)


# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted):
  print(f"{len(actual)}, {len(predicted)}")
  scaler = StandardScaler()
  scaler.fit(actual)
  predicted_not_normlized = scaler.inverse_transform(predicted)
  predicted_not_normlized_rounded = [np.round(num,2) for num in predicted_not_normlized]
  mse_score= mse(actual,predicted_not_normlized_rounded)
  rmse_score = sqrt(mse_score)
  mae_score = mae(actual,predicted_not_normlized_rounded)
  final_r2_score = r2_score(actual,predicted_not_normlized_rounded,multioutput='variance_weighted')
  evaluator = RegressionMetric(actual, predicted_not_normlized_rounded)
  mre_score = np.mean(evaluator.mean_relative_error())
  return mse_score, rmse_score,mae_score,mre_score,final_r2_score,predicted_not_normlized_rounded

# evaluate a single model
def evaluate_model(test_x,test_y,model):
  predictions = model.predict(test_x, verbose=0)
  mse_score, rmse_score,mae_score,mre_score,final_r2_score, y_hat_not_normlized = evaluate_forecasts(test_y, predictions)
  return mse_score, rmse_score,mae_score,mre_score,final_r2_score,y_hat_not_normlized

def summarize_scores2(name, mse_score, rmse_score,mae_score,mre_score,final_r2_score):
  return {  "Model_Name": name,
            "MAE": mae_score,
            "MRE": mre_score,
            "MSE": mse_score,
            "RMSE": rmse_score,
            "R2": final_r2_score}

### model

In [ ]:
from tensorflow.keras.optimizers import Nadam

In [ ]:
def get_gru_models_info_us(train_x,train_y,val_x,val_y,test_x,y_test,test_y_for_evaluation,company_name):
  # define parameters
  verbose, epochs, batch_size = 0, 20, math.floor(len(train_x)*0.05)
  n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
  # reshape output into [samples, timesteps, features]
  train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))
  # define model
  bench_gru = Sequential()
  bench_gru.add(GRU(256, activation='tanh', input_shape=(n_timesteps, n_features)))
  bench_gru.add(Dense(5))
  bench_gru.compile(loss='mse', optimizer=Nadam())
  tf.keras.utils.plot_model(bench_gru)
  # fit network
  bench_gru.fit(train_x,
          train_y,
          epochs=epochs,
          batch_size=batch_size,
          verbose=verbose,
          validation_data=(val_x,val_y))
  predictions = bench_gru.predict(test_x, verbose=0)
  mse_score_G_p, rmse_score_G_p,mae_score_G_p,mre_score,final_r2_score_G_p,predictions_G_p = evaluate_model(test_x,test_y_for_evaluation,bench_gru)
  print(f"{company_name} GRU(256)")
  #pickle.dump(bench_gru , open(f'/content/drive/MyDrive/KFUPM/Graduation/final_experiments/models/Bench/{company_name}_Benc_GRU_{256}.pkl' , 'wb'))
  #df_forecasts = pd.DataFrame(data = np.array(predictions_G_p).flatten() )
  #df_forecasts.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/models/Bench/{company_name}_Benc_forcasts_all.csv")
  print(summarize_scores2('GRU', mse_score_G_p, rmse_score_G_p,mae_score_G_p,mre_score,final_r2_score_G_p))




In [ ]:
def train_evalute_models_for_company_us(companies):
  for company_key in companies:
    dataset = pd.read_csv(f'{data_folder}{company_key}_selected_features.csv',index_col='Date',parse_dates=True)
    print(f'@@@@@@@@@@@@@@@@@@@@@@@_{company_key}_@@@@@@@@@@@@@@@@@@@@@@@@@@@')
    dataset = pd.read_csv(f'{data_folder}{company_key}_selected_features.csv',index_col='Date',parse_dates=True)
    company, company_selected_ti = get_company_df_bench_us(dataset,company_key)
    print(company_selected_ti)
    train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test = prepare_company_for_training_and_testing(company,company_selected_ti)
    test_y_for_evaluation = true_y_to_evaluation_form(test['Close'].values, 5, 5)
    get_gru_models_info_us(train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test_y_for_evaluation,company_key)

### Excution

In [ ]:
us = ["AAPL","BAC","GOOG","JNJ"]

In [ ]:
train_evalute_models_for_company_us(us)

@@@@@@@@@@@@@@@@@@@@@@@_AAPL_@@@@@@@@@@@@@@@@@@@@@@@@@@@
['Open', 'High', 'Low', 'Close', 'dema', 'hma', 'tema', 'wma', 'zelma']
Train shape: (5114, 9), Val shape: (366, 9), Test shape: (1160, 9)
*******************************************
Train data after multistep approach
X shape: (5104, 5, 9)
y shape: (5104, 5)
Validation multistep approach
X shape: (356, 5, 9)
y shape: (356, 5)
Test multistep approach
X shape: (1150, 5, 9)
y shape: (1150, 5)
1150, 1150
AAPL GRU(256)
{'Model_Name': 'GRU', 'MAE': 7.99259043945313, 'MRE': np.float64(0.18176799999999999), 'MSE': 91.50039041019426, 'RMSE': 9.56558364190049, 'R2': 0.2531035031437007}
@@@@@@@@@@@@@@@@@@@@@@@_BAC_@@@@@@@@@@@@@@@@@@@@@@@@@@@
['Open', 'High', 'Low', 'Close', 'dema', 'hma', 'tema', 'wma', 'zelma']
Train shape: (5114, 9), Val shape: (366, 9), Test shape: (1160, 9)
*******************************************
Train data after multistep approach
X shape: (5104, 5, 9)
y shape: (5104, 5)
Validation multistep approach
X shape: (356

## Combine the benchmark with our models' forcasts

In [ ]:
def prepare_testing_values(company_name):
  company =  pd.read_csv(f'{data_folder}{company_key}_selected_features.csv',index_col='Date',parse_dates=True)
  train,val, test =  company.loc[:'2015-12-31'],company.loc['1-1-2016':'31-12-2016'],company.loc['1-1-2017':]
  print(f'Train shape: {train.shape}, Val shape: {val.shape}, Test shape: {test.shape}')
  test_y_for_evaluation = np.array(true_y_to_evaluation_form(test['Close'].values, 5, 5)).flatten()
  return test_y_for_evaluation

In [ ]:
us = ["AAPL","BAC","GOOG","JNJ"]

In [ ]:
for company_key in us:
  print(f'@@@@@@@@@@@@@@@@@@@@@@@_{company_key}_@@@@@@@@@@@@@@@@@@@@@@@@@@@')
  models = pd.read_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/US/{company_key}_forcasts_all.csv")
  bench = pd.read_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/models/Bench/{company_key}_Benc_forcasts_all.csv")
  models["Benchmark"] = bench["0"]
  models["Actual"] = prepare_testing_values(company_key)
  models.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/US/{company_key}_forcasts_all.csv",index=False)



@@@@@@@@@@@@@@@@@@@@@@@_AAPL_@@@@@@@@@@@@@@@@@@@@@@@@@@@
Train shape: (5114, 34), Val shape: (366, 34), Test shape: (1160, 34)
@@@@@@@@@@@@@@@@@@@@@@@_BAC_@@@@@@@@@@@@@@@@@@@@@@@@@@@
Train shape: (5114, 34), Val shape: (366, 34), Test shape: (1160, 34)
@@@@@@@@@@@@@@@@@@@@@@@_GOOG_@@@@@@@@@@@@@@@@@@@@@@@@@@@
Train shape: (5114, 34), Val shape: (366, 34), Test shape: (1160, 34)
@@@@@@@@@@@@@@@@@@@@@@@_JNJ_@@@@@@@@@@@@@@@@@@@@@@@@@@@
Train shape: (5114, 34), Val shape: (366, 34), Test shape: (1160, 34)


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/Saudi/Forecasts/ADC_forcasts_all.csv")

In [ ]:
df.shape

(5750, 22)

In [ ]:
df.head()

,Baseline_Random Walk,BRNN,GRU,LSTM,"Avg(B,G,L)","Avg(B,G)","Avg(B,L)","Avg(G,L)","Sta_ANN(B,G,L)","Sta_ANN(B,G)",...,"Sta_GRU(B,G,L)","Sta_GRU(B,G)","Sta_GRU(B,L)","Sta_GRU(G,L)","Sta_LSTM(B,G,L)","Sta_LSTM(B,G)","Sta_LSTM(B,L)","Sta_LSTM(G,L)",Benchmark,Actual
0,13.389999,13.71,13.81,13.63,13.72,13.76,13.67,13.72,13.96,14.03,...,13.94,13.93,13.88,13.81,13.94,13.97,13.91,13.98,13.68,13.84
1,13.299999,13.77,13.96,13.68,13.81,13.87,13.73,13.82,13.99,14.02,...,13.94,13.95,13.87,13.91,13.94,13.96,13.95,13.98,14.41,13.84
2,13.209999,13.69,13.97,13.68,13.78,13.83,13.68,13.82,14.00,14.00,...,13.91,13.93,13.87,13.97,13.97,14.01,13.94,14.01,14.58,14.16
3,13.119999,13.76,14.00,13.76,13.84,13.88,13.76,13.88,14.00,14.03,...,13.95,14.00,13.91,13.93,13.94,14.04,13.95,13.97,14.65,12.75
4,13.029999,13.76,14.00,13.69,13.81,13.88,13.72,13.84,13.98,14.05,...,14.04,13.98,13.93,13.91,13.95,14.04,14.01,13.99,14.98,13.48
